## Install TensorFlow 1.14 (GPU variant) W&B

In [0]:
!pip install -q tensorflow-gpu==1.14.0
!pip install wandb

## Imports and installation verification

In [0]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from wandb.keras import WandbCallback
import tensorflow as tf
import numpy as np
import wandb
import time

In [0]:
tf.__version__

## W&B setup

In [0]:
!wandb login

In [4]:
wandb.init()
config = wandb.config

config.batch_size = 128
config.epochs=5

## Data collection and preprocessing

In [0]:
fashion_mnist = mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [0]:
x_train  = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000, 28, 28, 1))

In [7]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28, 1), (60000,), (10000, 28, 28, 1), (10000,))

## Model building and summarization

In [0]:
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2,2)),
        Conv2D(64, (3, 3), activation='relu')
    ])
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

In [9]:
model = create_model()
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
____

## Model training profiling

In [10]:
%%prun
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          epochs=config.epochs, batch_size=config.batch_size, 
          callbacks=[WandbCallback()], verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 54s 901us/sample - loss: 0.6179 - acc: 0.7748 - val_loss: 0.4529 - val_acc: 0.8347
Epoch 2/5
60000/60000 [==============================] - 54s 908us/sample - loss: 0.3854 - acc: 0.8613 - val_loss: 0.3699 - val_acc: 0.8687
Epoch 3/5
60000/60000 [==============================] - 56s 930us/sample - loss: 0.3329 - acc: 0.8796 - val_loss: 0.3479 - val_acc: 0.8723
Epoch 4/5
60000/60000 [==============================] - 54s 898us/sample - loss: 0.3000 - acc: 0.8910 - val_loss: 0.3210 - val_acc: 0.8848
Epoch 5/5
60000/60000 [==============================] - 54s 896us/sample - loss: 0.2775 - acc: 0.8982 - val_loss: 0.3028 - val_acc: 0.8934
 

Here are the major logs from the profiling:

```
3326560 function calls (3256171 primitive calls) in 272.912 seconds

   Ordered by: internal time
   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2740  259.207    0.095  259.207    0.095 {built-in method _pywrap_tensorflow_internal.TF_SessionRunCallable}
    31609    2.641    0.000    2.641    0.000 {method 'acquire' of '_thread.lock' objects}
     2740    1.945    0.001    1.945    0.001 generic_utils.py:531(<listcomp>)
    34989    0.777    0.000    0.778    0.000 {built-in method numpy.array}
    16429    0.558    0.000    0.558    0.000 socket.py:333(send)
      6/1    0.451    0.075  272.567  272.567 training_arrays.py:45(model_iteration)
     2740    0.313    0.000  262.203    0.096 backend.py:3250(__call__)
     2345    0.273    0.000    4.382    0.002 generic_utils.py:353(update)
 ```

CPU and Memory profilings can be found [here](https://app.wandb.ai/sayakpaul/uncategorized/runs/5d54sfb1). Let's now do the *exact* same thing with TensorFlow 2.0. 